In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [41]:
class PDF(object):
    def __init__(self, pdf, size=(200,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

Discussion with Hong

The peaks center not obvious on the delta heatmap, so we using the real center of the peaks;

Do not plot the peak region and just + - 5k

# Tester the reference point does not having the same outputs as the 1.2_peak.pdf

1_2.peak.pdf using the coup bed file to plot the heatmap, which has the phenomena that the signal spans across the whole region

while using the delta heatmap the results does not show such pattern

Correspondence:
The 1_2.peak.pdf have wide signal spanning across +- 5k due to the plot are stretched vertically and the color box becomes larger
Another reason is the bin size; small binsize having the small signal values

## 1.1 Use the union of the T1 WT to do the heatmap plot

In [19]:
ctrl = 'ENL'
treats = ['T1', 'T2', 'T3', 'Y78A']
# sigpath: store the previous generated merged peaks signal for T and wt
sigpath = './res_avg_signal/' 
# outpath = './res_delta_heatmap/'
outpath = './res_delta_heatmap/peak_center_no_flank_5k/'
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
treat = treats[0]

In [38]:
    sigfile = f'{sigpath}293_{treat}_WT.merge.sig.anno.pval.txt'
    line = "awk \'BEGIN{OFS=\"\\t\"}{if (NR > 1) print $1, $2, $3,NR-2,\".\", \"+\"}\'" + f' {sigfile} > {outpath}{treat}.bed'
    print(line + '\n****')
    os.system(line)
    bw_files = [
            glob.glob(bwpath + exp + '*' + tmp + '*.bw')[0]
            for tmp in [treat, ctrl]
    ]
    bw_files_format = len(bw_files) * "{} "
    bw_files_format = bw_files_format.format(*bw_files)
    bed_file_format = f'{outpath}{treat}.bed'
    command = "computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R {0} -S {1} --skipZeros -o {2}.scale.gz --outFileNameMatrix {3}.scale.txt ".format(
    bed_file_format, bw_files_format,
    outpath + exp + '_' + treat + '_' + ctrl,
    outpath + exp + '_' + treat + '_' + ctrl)
    print(command + '\n\n')
    os.system(command)
#     os.system(
#         'rm {}.scale.gz'.format(outpath + exp + '_' + treat + '_' + ctrl))     
    line = f'plotHeatmap --sortUsingSamples 1 -m {outpath}293_T1_ENL.scale.gz -out {outpath}test.peak.pdf \
    --outFileNameMatrix test.mat --colorMap Blues Blues  --dpi 300 --refPointLabel center'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR > 1) print $1, $2, $3,NR-2,".", "+"}' ./res_avg_signal/293_T1_WT.merge.sig.anno.pval.txt > ./res_delta_heatmap/peak_center_no_flank_5k/T1.bed
****
computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R ./res_delta_heatmap/peak_center_no_flank_5k/T1.bed -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw  --skipZeros -o ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.gz --outFileNameMatrix ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.txt 


plotHeatmap --sortUsingSamples 1 -m ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.gz -out ./res_delta_heatmap/peak_center_no_flank_5k/test.peak.pdf --outFileNameMatrix test.mat --colorMap Blues Blues  --dpi 300 --refPointLabel center


0

In [37]:
PDF(f'{outpath}test.peak.pdf',size=(300,250))

#### 1.2 Use the different bin size to plot the co up gene list

In [43]:
coup_path = '/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks/'
ths = [1.2, 1.5]
names = ['1_2','1_5']

In [52]:
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
for name in names:
    outBed = f'{coup_path}293.all.{name}.heat.bed'
    bw_files_format = ' '.join(bwfiles)
    outFile = f'{outpath}{name}'
    line = f'~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S {bw_files_format} -R {outBed} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 50 --skipZeros --missingDataAsZero -o {outFile}.Ts.ENL.peak.gz -p 8'
    print(line)
    os.system(line)

~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw -R /home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks/293.all.1_2.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 50 --skipZeros --missingDataAsZero -o ./res_delta_heatmap/peak_center_no_flank_5k/1_2.Ts.ENL.peak.gz -p 8
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw /home/xc3/output/ENL2/

In [48]:
!ls $outpath

1_2.Ts.ENL.peak.gz  293_T1_ENL.scale.gz   T1.bed	       test.peak.pdf
1_5.Ts.ENL.peak.gz  293_T1_ENL.scale.txt  name.Ts.ENL.peak.gz


In [55]:
# plot the heatmap
for name in names:
    line = f'plotHeatmap --sortUsingSamples 1 -m {outpath}{name}.Ts.ENL.peak.gz -out {outpath}{name}.peak.pdf --outFileNameMatrix {outpath}{name}.txt --colorMap Blues Blues Blues Blues Blues --dpi 300 --refPointLabel center --heatmapHeight 5'
    print(line)
    os.system(line)

plotHeatmap --sortUsingSamples 1 -m ./res_delta_heatmap/peak_center_no_flank_5k/1_2.Ts.ENL.peak.gz -out ./res_delta_heatmap/peak_center_no_flank_5k/1_2.peak.pdf --outFileNameMatrix ./res_delta_heatmap/peak_center_no_flank_5k/1_2.txt --colorMap Blues Blues Blues Blues Blues --dpi 300 --refPointLabel center --heatmapHeight 5
plotHeatmap --sortUsingSamples 1 -m ./res_delta_heatmap/peak_center_no_flank_5k/1_5.Ts.ENL.peak.gz -out ./res_delta_heatmap/peak_center_no_flank_5k/1_5.peak.pdf --outFileNameMatrix ./res_delta_heatmap/peak_center_no_flank_5k/1_5.txt --colorMap Blues Blues Blues Blues Blues --dpi 300 --refPointLabel center --heatmapHeight 5


In [56]:
PDF(f'{outpath}{name}.peak.pdf')

# 2. Re-generate the delta heatmap with binsize 100


## 2.1 Use computeMatrix to extract the binning signal for the corresponding bed file

In [58]:
for treat in treats:
    #1. extract the first three columns as the bed file
    sigfile = f'{sigpath}293_{treat}_WT.merge.sig.anno.pval.txt'
    line = "awk \'BEGIN{OFS=\"\\t\"}{if (NR > 1) print $1, $2, $3,NR-2,\".\", \"+\"}\'" + f' {sigfile} > {outpath}{treat}.bed'
    print(line + '\n****')
    os.system(line)
    # 2. use the bed file to extract the binning signal (center -5k~5k)
    bw_files = [
            glob.glob(bwpath + exp + '*' + tmp + '*.bw')[0]
            for tmp in [treat, ctrl]
    ]
    bw_files_format = len(bw_files) * "{} "
    bw_files_format = bw_files_format.format(*bw_files)
    bed_file_format = f'{outpath}{treat}.bed'
    command = "computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R {0} -S {1} --skipZeros -o {2}.scale.gz --outFileNameMatrix {3}.scale.txt ".format(
    bed_file_format, bw_files_format,
    outpath + exp + '_' + treat + '_' + ctrl,
    outpath + exp + '_' + treat + '_' + ctrl)
    print(command + '\n\n')
    os.system(command)
    os.system(
        'rm {}.scale.gz'.format(outpath + exp + '_' + treat + '_' + ctrl))  

awk 'BEGIN{OFS="\t"}{if (NR > 1) print $1, $2, $3,NR-2,".", "+"}' ./res_avg_signal/293_T1_WT.merge.sig.anno.pval.txt > ./res_delta_heatmap/peak_center_no_flank_5k/T1.bed
****
computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R ./res_delta_heatmap/peak_center_no_flank_5k/T1.bed -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw  --skipZeros -o ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.gz --outFileNameMatrix ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.txt 


awk 'BEGIN{OFS="\t"}{if (NR > 1) print $1, $2, $3,NR-2,".", "+"}' ./res_avg_signal/293_T2_WT.merge.sig.anno.pval.txt > ./res_delta_heatmap/peak_center_no_flank_5k/T2.bed
****
computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R ./res_delta_heatmap/peak_center_no_flank_5k/T2.bed -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /ho

## 2.2 split the scale binning signal file to upper, lower, no diff
scale file is the binning signal

In [59]:
fc = 1.2
for treat in treats:
    sigfile = f'{sigpath}293_{treat}_WT.merge.sig.anno.pval.txt'
    df = pd.read_csv(sigfile,sep='\t',header=0)  
    print('preprocess on {}'.format(sigfile))
    exts = ['upper', 'lower', 'nodiff']
    ind_up = df.iloc[:,4].div(df.iloc[:,5]) >= fc
    ind_down = df.iloc[:,4].div(df.iloc[:,5]) <= (1.0/fc)
    ind_no = (df.iloc[:,4].div(df.iloc[:,5]) > 1.0/fc) & (df.iloc[:,4].div(df.iloc[:,5]) < fc)
    inds = [ind_up, ind_down, ind_no]
    # the extracted binning file 
    scalefile = '{}.scale.txt'.format(outpath + exp + '_' + treat + '_' + ctrl)
    df_scale = pd.read_csv(scalefile, sep='\t', header=2)
    # remove the first colname, which denotes how many peaks
    colnames = df_scale.columns.values[1:]
    df_scale = df_scale.iloc[:, :-1]
    df_scale.columns = colnames
    
    for ind, ext in zip(inds, exts):
        print(ext + '***\n')
        df_tmp = df[ind].copy()
        treat_id = [
            i for i, ele in enumerate(df_tmp.columns) if treat in ele
        ][0]
        df_tmp.sort_values(
            by=df_tmp.columns[treat_id], inplace=True)  #4: treat
        #3. split the df_tmp into three groups for treat and ctrl and delta
        inds = df_tmp.index
        #3.1 treat: # add 1
        treat_id = [i for i, ele in enumerate(colnames) if treat in ele]
        df_treat = df_scale.iloc[inds, treat_id]
        df_treat.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, treat),
            index=False)
        #3.2 ctrl:
        ctrl_id = [i for i, ele in enumerate(colnames) if ctrl in ele]
        df_ctrl = df_scale.iloc[inds, ctrl_id]
        df_ctrl.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, ctrl),
            index=False)
        #3.3 delta verified
        df_delta = pd.DataFrame(df_treat.values - df_ctrl.values)
        df_delta.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, 'delta'),
            index=False)

preprocess on ./res_avg_signal/293_T1_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***

preprocess on ./res_avg_signal/293_T2_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***

preprocess on ./res_avg_signal/293_T3_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***

preprocess on ./res_avg_signal/293_Y78A_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***



# 3. The delta heatmap for 1.2 coup peaks

previous created delta heatmap by jiejun has wider signal spread
Another reason is that when we plot the tss region, lots of genes shared the same region, so the color box is huge

In [60]:
workPath = '/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks/'
os.chdir(workPath)
!pwd

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks


In [67]:
#format the bed file into the bed file add three columns more; 
names = ['1_2']
for name in names:
    Bed = f'293.all.{name}.sig.anno.txt'
    outBed = f'293.all.{name}.new.heat.bed'
    line = "awk 'BEGIN{OFS=\"\\t\"}{if (NR>1){print $1,$2,$3,NR-2,\".\",\"+\"}}'" + f' {Bed} > {outBed}'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,NR-2,".","+"}}' 293.all.1_2.sig.anno.txt > 293.all.1_2.new.heat.bed


## for each pair sample extract the binning signal

In [92]:
samples = treats + [ctrl]
for sample in samples:
    bwfile = glob.glob(bwpath + exp + '*' + sample + '*.bw')[0]
    outBed = "293.all.1_2.new.heat.bed"
    print(bwfile)
    line = f'~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S {bwfile} -R {outBed} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_{sample}_co-UP_genes.gz -p 8'
    print(line + '\n****\n')
    os.system(line)

/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw -R 293.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_T1_co-UP_genes.gz -p 8
****

/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw -R 293.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_T2_co-UP_genes.gz -p 8
****

/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw -R 293.all.1_2.new.heat.bed --beforeRegionStartLen

chr15	96872571	96885658	0	.	+
chr7	5457629	5470124	1	.	+
chr6	1608510	1616027	2	.	+
chr17	79368489	79375603	3	.	+
chr12	12869276	12879696	4	.	+
chr11	65186288	65204652	5	.	+
chr7	27197889	27240775	6	.	+
chr11	65264641	65277345	7	.	+
chr18	19747704	19753749	8	.	+
chr6	32934633	32953286	9	.	+
/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks
